# Exploration

In [1]:
import csv
import re
from collections import Counter
from itertools import islice
from tqdm import tqdm_notebook

In [2]:
# Params
DATA = '../data/didascalies.csv'

GROUPS = [
    'larem',
    'rem',
    'modem',
    'lr',
    'lfi',
    'fi',
    'lc',
    'udi',
    'ng',
    'gdr',
    'mda'
]

GROUPS = [(g, re.compile('\\b' + g + '\\b')) for g in GROUPS]

In [3]:
# Helpers
def is_didascalie(line):
    return line['nom'] == 'NULL' and line['parlementaire'] == 'NULL'

def is_relevant(intervention):
    intervention = intervention.lower()
    
    return (
        'applaud' in intervention or
        'murmur' in intervention or
        'rire' in intervention or
        'banc' in intervention
    )

def extract_groups(intervention):
    intervention = intervention.lower()
    
    if 'bancs' not in intervention:
        return None
    
    groups = [g for g, pattern in GROUPS if pattern.search(intervention)]
    
    return groups

In [4]:
COUNTER = Counter()
with open(DATA, 'r') as f:
    reader = csv.DictReader(f)
    
    for line in tqdm_notebook(reader):
        if not is_didascalie(line):
            continue
            
        groups = extract_groups(line['intervention'])

        if groups:
            for g in groups:
                COUNTER[g] += 1

In [5]:
COUNTER.most_common()

[('rem', 6106),
 ('lr', 5523),
 ('modem', 4354),
 ('fi', 3806),
 ('larem', 3374),
 ('gdr', 2318),
 ('ng', 1660),
 ('lc', 782),
 ('udi', 776),
 ('lfi', 1)]